In [1]:
""" This is the main script for eap ml project, which plays with all ml methods, applicable for cpcrsp analysis
Created on Sat Feb 12 15:19:54 2022
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import random
import gc
import dill
import statsmodels.api as sm
from sklearn.svm import SVR


from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNetCV
from sklearn import svm
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

from xgboost import XGBRegressor

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

plt.style.use('seaborn-white')
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', 40)
gc.enable()

In [12]:
path = '../input/mleap-data/'

tempx = pd.read_csv(path + 's_train_X.csv')
tempy = pd.read_csv(path + 's_train_y.csv')
#tempx.drop(columns='Unnamed: 0', inplace=True)
#tempy.drop(columns='Unnamed: 0', inplace=True)
sp_train_X = np.array(tempx)
s_train_y = np.array(tempy)
print(sp_train_X.shape,s_train_y.shape)

(1257579, 28) (1257579, 2)


In [14]:
tempx = pd.read_csv(path + 's_train_X.csv')
tempy = pd.read_csv(path + 's_train_y.csv')
tempx

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,0,-0.243009,-1.074712,0.990305,-4.326325,-3.363023,-1.616348,-0.101192,-1.090530,0.074331,-1.246267,-0.898679,-1.062922,-1.310861,-0.364439,-0.273374,-2.276979,1.280535,-1.183398,-1.594133,0.358283,-2.035241,0.080993,1.144527,-1.253556,0.578784,-0.263684,1.906464
1,1,-0.243009,-0.971339,0.990305,-4.326325,-3.363023,-1.616348,-1.486989,-1.091612,0.074331,0.533935,-0.869110,-1.062481,0.506767,-0.226494,-0.273374,-2.374758,1.280535,-1.183398,-1.594133,0.358283,-2.035241,0.080993,-1.312468,-1.213550,0.456332,-0.263684,-0.098645
2,2,-0.243009,-1.057003,0.990305,-4.326325,-3.363023,-1.616348,-1.826523,-0.896354,0.074331,1.130247,-0.644257,-1.062628,1.527008,-0.266564,-0.273374,-2.500734,1.280535,-1.183398,-1.594133,0.358283,-2.215931,0.080993,1.369124,-1.223882,0.481324,-0.263684,-2.433507
3,3,-0.243009,-1.102543,0.990305,-4.326325,-3.363023,-1.616348,-0.099809,-1.191436,0.074331,-1.246267,-1.437214,-1.062334,-1.311134,-0.526218,-0.273374,-2.500734,1.280535,-1.183398,-1.594133,0.358283,-2.215931,0.080993,0.947485,-0.650079,0.152906,-0.263684,-0.100045
4,4,-0.243009,-0.985617,0.990305,-4.326325,-3.363023,-1.616348,2.204548,-1.192405,0.074331,1.631517,-1.016956,3.831617,1.718698,-0.121037,-0.273374,-2.374758,1.280535,-1.183398,-1.594133,0.358283,-2.215931,0.080993,-1.312468,-0.716609,0.107574,-0.263684,-0.097246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257574,1257574,-0.090733,0.518169,-2.302581,-0.713413,-1.111884,0.339176,-0.967385,-0.081830,-3.171429,-0.335738,1.243210,-0.462838,-0.190681,-0.353712,-0.570340,2.655980,-1.680829,-0.604873,-1.208066,4.040666,2.747210,-3.140100,-0.039642,0.442993,-0.607330,-0.613075,-0.800385
1257575,1257575,-0.138680,0.310771,-2.302581,-0.713413,-1.111884,0.339176,0.017790,-0.500201,-3.176602,0.834343,1.281042,1.318033,0.224715,-0.202460,-0.583245,2.661122,-1.680829,-0.604873,-1.208066,4.040666,2.789784,-3.158960,-0.364575,0.468002,-0.573493,-0.615193,0.637974
1257576,1257576,-0.118931,0.335742,-2.302581,-0.713413,-1.111884,0.339176,-0.908813,-0.413949,-3.174678,0.746594,1.223230,0.182645,0.535030,-0.104845,-0.582832,2.606503,-1.680829,-0.604873,-1.208066,4.040666,2.773864,-3.192468,-0.417016,0.788133,-0.563579,-0.615193,-0.354678
1257577,1257577,-0.030183,0.659125,-2.302581,-0.713413,-1.111884,0.339176,1.828903,-0.575448,-3.160503,1.102001,1.028767,1.488074,0.692212,0.097405,-0.577758,2.712985,-1.680829,-0.604873,-1.208066,4.040666,2.761317,-3.228561,-0.435491,0.804901,-0.562638,-0.615193,-0.259998


In [11]:
parameters = {'learning_rate': [0.01, 0.02, 0.03], 'subsample':[0.2, 0.3],
             'n_estimators':[100, 200, 300, 500]}

grid_search = GridSearchCV(estimator=XGBRegressor(tree_method='gpu_hist', gpu_id=0,
                                                  max_depth=2,
        colsample_bytree=0.5, nthread=4, seed=42), param_grid=parameters, n_jobs =-1, cv = 2, verbose=True)

grid_search.fit(sp_train_X, s_train_y)
print(grid_search.best_score_, grid_search.best_params_)

r2_score(s_train_y, grid_search.predict(sp_train_X))

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   19.7s finished


0.0060913534919707635 {'learning_rate': 0.02, 'n_estimators': 500, 'subsample': 0.3}


0.019223597715924412

In [ ]:
bt = XGBRegressor(n_estimators=1000, max_depth=2, eta=0.1, subsample=0.6, colsample_bytree=0.5, nthread=4)
bt.fit(sp_train_X, s_train_y)

[r2_score(s_train_y, bt.predict(sp_train_X)), ]

In [ ]:
grid_values = [{'max_depth': [8,10,12,15], 'max_features': list(np.arange(0.4,0.61,0.2))}]
rfc = RandomForestRegressor(random_state=42)
model_rf = GridSearchCV(rfc, grid_values, cv = 10)
model_rf.fit(sp_train_X, s_train_y)
print(model_rf.best_score_, model_rf.best_params_)